In [1]:
# Import dependencies
#from api_key import api_key
from jsonschema import validate
import json
import requests
import pandas as pd

api_key = 'KKB4LAOR9vVH9_LldMfXSwCzLRcRkmB_BGf_lCbgO-l2VstSuhbSLxX8HQXGGSHZC-ZVKqy5VAzK1zT6bMkhrRtQ9dGs9HWaO77SP8pvSINjYw53tGI7hIviv2puZXYx'

In [2]:
# Set up Yelp API constants
API_HOST = 'https://api.yelp.com/v3/businesses/search'
HEADERS = {
    'Authorization': 'bearer %s' % api_key
}

# Schema for comparing in checking before extraction
schema = {
    'alias': '',
    'categories': [],
    'coordinates': {},
    'display_phone': '',
    'distance': 0.00,
    'id': 'string',
    'image_url': '',
    'is_closed': True,
    'location': {},
    'name': '',
    'phone': '',
    'price': '',
    'rating': 0.0,
    'review_count': 0,
    'transaction': [],
    'url': ''
}

In [3]:
# Simple request function for bussiness search endpoint from Yelp API
def request(term = '', limit = 0, rad = 0, loc = ''):
    # Create the parameters for search
    params = {
        'term': term,
        'limit': limit,
        'radius': rad,
        'location': loc
    }
    
    # Send the request
    response = requests.get(API_HOST, headers = HEADERS, params = params)
    
    # Verify the response and return None if error returned else return the json data
    if 'error' in response.json().keys():
        return None
    else:
        return response.json()

# Function to verify the predefined schema on what we should be expecting before extracting
def verify_schema(data = None):
    
    # Verify the object entered before extraction
    if data == None:
        return False
    elif not isinstance(data, dict):
        return False
    else:
        try:
            validate(instance=data['businesses'][0], schema=schema)
            return True
        except:
            return False

# Function to extract the id, name, price, rating, review_count, location (address 1, address 2, address 3, city, 
# state, zip_code), coordinates (latitude and longtitude), and phone into a DataFrame
def json_to_dataframe(data = None):
    if not verify_schema(data):
        return None
    else:
        data_df = pd.DataFrame(data['businesses'])
        return data_df

In [4]:
test_starbuck = request('starbucks', limit = 50, rad = 1000, loc = 'NYC')

startbucks_df = json_to_dataframe(test_starbuck)
startbucks_df.head()
#startbucks_df.to_csv('starbucks.csv', index = True)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,qcnoyytlFIuqlcjDXkXJiw,starbucks-brooklyn-21,Starbucks,https://s3-media2.fl.yelpcdn.com/bphoto/rvbmfb...,False,https://www.yelp.com/biz/starbucks-brooklyn-21...,80,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",2.5,"{'latitude': 40.70275444, 'longitude': -73.990...",[delivery],$$,"{'address1': '67 Main St', 'address2': '', 'ad...",+17188550856,(718) 855-0856,414.846802
1,60agfQbky4cX8BEApyltIA,starbucks-new-york-524,Starbucks,https://s3-media4.fl.yelpcdn.com/bphoto/ZM_sCP...,False,https://www.yelp.com/biz/starbucks-new-york-52...,36,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",2.0,"{'latitude': 40.71102, 'longitude': -74.00091}",[delivery],$,"{'address1': '375 Pearl St', 'address2': '', '...",+16466999983,(646) 699-9983,833.889269
2,Y_kNzTJJUx9aFj1t_DVWkA,starbucks-surrey-6,Starbucks,,False,https://www.yelp.com/biz/starbucks-surrey-6?ad...,3,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",5.0,"{'latitude': 49.1186981201172, 'longitude': -1...",[],$,"{'address1': '199 Water Street', 'address2': '...",,,876.341256
3,C0EKm9V9QI2R47QkRWA5aQ,starbucks-new-york-290,Starbucks,https://s3-media3.fl.yelpcdn.com/bphoto/Rvok-B...,False,https://www.yelp.com/biz/starbucks-new-york-29...,44,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",3.0,"{'latitude': 40.7049014, 'longitude': -74.0072...",[],$,"{'address1': '99 Wall St', 'address2': '', 'ad...",+19299551841,(929) 955-1841,1093.379283
4,OuNYQaqEJjkBHRdnC-4HOw,starbucks-new-york-381,Starbucks,https://s3-media2.fl.yelpcdn.com/bphoto/SV8ynJ...,False,https://www.yelp.com/biz/starbucks-new-york-38...,40,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",2.5,"{'latitude': 40.708413, 'longitude': -74.007387}",[delivery],$$,"{'address1': '100 William St', 'address2': '',...",+12125099709,(212) 509-9709,1151.989274


In [ ]:
"""
db:

shop
> ID (str) PK
> name (str)
> review_count (int)
> rating (float)
> price (str)
> price_point (int) 
> address_Id (int) FK

address
> id (index) PK
> address1 (str)
> address2 (str)
> address3 (str)
> city (str)
> zip (str)
> country (str)
> state (str)
> lon (double)
> lat (double)

> pull dataset
> json > DF > extraction
> storing db
	if db exists > delete > recreate
	append data to respective tables
> d3.js > postgreSQL for data
	> csv

"""

In [5]:
startbucks_df.dtypes

id                object
alias             object
name              object
image_url         object
is_closed           bool
url               object
review_count       int64
categories        object
rating           float64
coordinates       object
transactions      object
price             object
location          object
phone             object
display_phone     object
distance         float64
dtype: object

In [6]:
startbucks_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,qcnoyytlFIuqlcjDXkXJiw,starbucks-brooklyn-21,Starbucks,https://s3-media2.fl.yelpcdn.com/bphoto/rvbmfb...,False,https://www.yelp.com/biz/starbucks-brooklyn-21...,80,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",2.5,"{'latitude': 40.70275444, 'longitude': -73.990...",[delivery],$$,"{'address1': '67 Main St', 'address2': '', 'ad...",+17188550856,(718) 855-0856,414.846802
1,60agfQbky4cX8BEApyltIA,starbucks-new-york-524,Starbucks,https://s3-media4.fl.yelpcdn.com/bphoto/ZM_sCP...,False,https://www.yelp.com/biz/starbucks-new-york-52...,36,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",2.0,"{'latitude': 40.71102, 'longitude': -74.00091}",[delivery],$,"{'address1': '375 Pearl St', 'address2': '', '...",+16466999983,(646) 699-9983,833.889269
2,Y_kNzTJJUx9aFj1t_DVWkA,starbucks-surrey-6,Starbucks,,False,https://www.yelp.com/biz/starbucks-surrey-6?ad...,3,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",5.0,"{'latitude': 49.1186981201172, 'longitude': -1...",[],$,"{'address1': '199 Water Street', 'address2': '...",,,876.341256
3,C0EKm9V9QI2R47QkRWA5aQ,starbucks-new-york-290,Starbucks,https://s3-media3.fl.yelpcdn.com/bphoto/Rvok-B...,False,https://www.yelp.com/biz/starbucks-new-york-29...,44,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",3.0,"{'latitude': 40.7049014, 'longitude': -74.0072...",[],$,"{'address1': '99 Wall St', 'address2': '', 'ad...",+19299551841,(929) 955-1841,1093.379283
4,OuNYQaqEJjkBHRdnC-4HOw,starbucks-new-york-381,Starbucks,https://s3-media2.fl.yelpcdn.com/bphoto/SV8ynJ...,False,https://www.yelp.com/biz/starbucks-new-york-38...,40,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",2.5,"{'latitude': 40.708413, 'longitude': -74.007387}",[delivery],$$,"{'address1': '100 William St', 'address2': '',...",+12125099709,(212) 509-9709,1151.989274
5,mEMPhPK6dSgy5eXS-kYHLg,starbucks-brooklyn-17,Starbucks,https://s3-media4.fl.yelpcdn.com/bphoto/IsyCZQ...,False,https://www.yelp.com/biz/starbucks-brooklyn-17...,69,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",3.0,"{'latitude': 40.694582, 'longitude': -73.993978}",[delivery],$$,"{'address1': '134 Montague St', 'address2': ''...",+17182430455,(718) 243-0455,1208.186328
6,t03ZbaLQP3MrMuUGb6MQdw,starbucks-new-york-506,Starbucks,https://s3-media3.fl.yelpcdn.com/bphoto/XApF9R...,False,https://www.yelp.com/biz/starbucks-new-york-50...,53,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",2.0,"{'latitude': 40.704653, 'longitude': -74.009001}",[delivery],$$,"{'address1': '110 Pearl St', 'address2': '', '...",+12124826530,(212) 482-6530,1242.904106
7,pTv4RDll3N_CGG3BwtEctg,starbucks-new-york-395,Starbucks,https://s3-media1.fl.yelpcdn.com/bphoto/ClYnav...,False,https://www.yelp.com/biz/starbucks-new-york-39...,59,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",3.0,"{'latitude': 40.71168566, 'longitude': -74.006...",[delivery],$,"{'address1': '38 Park Row', 'address2': '', 'a...",+12126088073,(212) 608-8073,1252.899146
8,j6nTfbYavGVsJKxz6bte6w,starbucks-new-york-214,Starbucks,https://s3-media3.fl.yelpcdn.com/bphoto/MdLapQ...,False,https://www.yelp.com/biz/starbucks-new-york-21...,61,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",3.5,"{'latitude': 40.71022348, 'longitude': -74.007...",[delivery],$,"{'address1': '130 Fulton St', 'address2': '', ...",+12129620439,(212) 962-0439,1269.573946
9,XzAZVxmGPcysK8hYdcAJKA,starbucks-new-york-229,Starbucks,https://s3-media2.fl.yelpcdn.com/bphoto/ms4p2K...,False,https://www.yelp.com/biz/starbucks-new-york-22...,53,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",2.5,"{'latitude': 40.71588714, 'longitude': -74.003...",[delivery],$$,"{'address1': '111 Worth St', 'address2': '', '...",+12129648846,(212) 964-8846,1379.974973


In [7]:
startbucks_df.name = startbucks_df.name.apply(lambda x: x.lower())
startbucks_df2 = startbucks_df.loc[startbucks_df['name'] == 'starbucks']
startbucks_df2

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,qcnoyytlFIuqlcjDXkXJiw,starbucks-brooklyn-21,starbucks,https://s3-media2.fl.yelpcdn.com/bphoto/rvbmfb...,False,https://www.yelp.com/biz/starbucks-brooklyn-21...,80,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",2.5,"{'latitude': 40.70275444, 'longitude': -73.990...",[delivery],$$,"{'address1': '67 Main St', 'address2': '', 'ad...",+17188550856,(718) 855-0856,414.846802
1,60agfQbky4cX8BEApyltIA,starbucks-new-york-524,starbucks,https://s3-media4.fl.yelpcdn.com/bphoto/ZM_sCP...,False,https://www.yelp.com/biz/starbucks-new-york-52...,36,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",2.0,"{'latitude': 40.71102, 'longitude': -74.00091}",[delivery],$,"{'address1': '375 Pearl St', 'address2': '', '...",+16466999983,(646) 699-9983,833.889269
2,Y_kNzTJJUx9aFj1t_DVWkA,starbucks-surrey-6,starbucks,,False,https://www.yelp.com/biz/starbucks-surrey-6?ad...,3,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",5.0,"{'latitude': 49.1186981201172, 'longitude': -1...",[],$,"{'address1': '199 Water Street', 'address2': '...",,,876.341256
3,C0EKm9V9QI2R47QkRWA5aQ,starbucks-new-york-290,starbucks,https://s3-media3.fl.yelpcdn.com/bphoto/Rvok-B...,False,https://www.yelp.com/biz/starbucks-new-york-29...,44,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",3.0,"{'latitude': 40.7049014, 'longitude': -74.0072...",[],$,"{'address1': '99 Wall St', 'address2': '', 'ad...",+19299551841,(929) 955-1841,1093.379283
4,OuNYQaqEJjkBHRdnC-4HOw,starbucks-new-york-381,starbucks,https://s3-media2.fl.yelpcdn.com/bphoto/SV8ynJ...,False,https://www.yelp.com/biz/starbucks-new-york-38...,40,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",2.5,"{'latitude': 40.708413, 'longitude': -74.007387}",[delivery],$$,"{'address1': '100 William St', 'address2': '',...",+12125099709,(212) 509-9709,1151.989274
5,mEMPhPK6dSgy5eXS-kYHLg,starbucks-brooklyn-17,starbucks,https://s3-media4.fl.yelpcdn.com/bphoto/IsyCZQ...,False,https://www.yelp.com/biz/starbucks-brooklyn-17...,69,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",3.0,"{'latitude': 40.694582, 'longitude': -73.993978}",[delivery],$$,"{'address1': '134 Montague St', 'address2': ''...",+17182430455,(718) 243-0455,1208.186328
6,t03ZbaLQP3MrMuUGb6MQdw,starbucks-new-york-506,starbucks,https://s3-media3.fl.yelpcdn.com/bphoto/XApF9R...,False,https://www.yelp.com/biz/starbucks-new-york-50...,53,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",2.0,"{'latitude': 40.704653, 'longitude': -74.009001}",[delivery],$$,"{'address1': '110 Pearl St', 'address2': '', '...",+12124826530,(212) 482-6530,1242.904106
7,pTv4RDll3N_CGG3BwtEctg,starbucks-new-york-395,starbucks,https://s3-media1.fl.yelpcdn.com/bphoto/ClYnav...,False,https://www.yelp.com/biz/starbucks-new-york-39...,59,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",3.0,"{'latitude': 40.71168566, 'longitude': -74.006...",[delivery],$,"{'address1': '38 Park Row', 'address2': '', 'a...",+12126088073,(212) 608-8073,1252.899146
8,j6nTfbYavGVsJKxz6bte6w,starbucks-new-york-214,starbucks,https://s3-media3.fl.yelpcdn.com/bphoto/MdLapQ...,False,https://www.yelp.com/biz/starbucks-new-york-21...,61,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",3.5,"{'latitude': 40.71022348, 'longitude': -74.007...",[delivery],$,"{'address1': '130 Fulton St', 'address2': '', ...",+12129620439,(212) 962-0439,1269.573946
9,XzAZVxmGPcysK8hYdcAJKA,starbucks-new-york-229,starbucks,https://s3-media2.fl.yelpcdn.com/bphoto/ms4p2K...,False,https://www.yelp.com/biz/starbucks-new-york-22...,53,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",2.5,"{'latitude': 40.71588714, 'longitude': -74.003...",[delivery],$$,"{'address1': '111 Worth St', 'address2': '', '...",+12129648846,(212) 964-8846,1379.974973


In [ ]:
#pd.json_normalize(startbucks_df2['coordinates'])

In [8]:
startbucks_df2[['latitude', 'longitude']] = pd.json_normalize(startbucks_df2['coordinates'])
startbucks_df2

C:\Users\yoeri\AppData\Local\Temp\ipykernel_28936\3661593508.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  startbucks_df2[['latitude', 'longitude']] = pd.json_normalize(startbucks_df2['coordinates'])
C:\Users\yoeri\AppData\Local\Temp\ipykernel_28936\3661593508.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  startbucks_df2[['latitude', 'longitude']] = pd.json_normalize(startbucks_df2['coordinates'])


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance,latitude,longitude
0,qcnoyytlFIuqlcjDXkXJiw,starbucks-brooklyn-21,starbucks,https://s3-media2.fl.yelpcdn.com/bphoto/rvbmfb...,False,https://www.yelp.com/biz/starbucks-brooklyn-21...,80,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",2.5,"{'latitude': 40.70275444, 'longitude': -73.990...",[delivery],$$,"{'address1': '67 Main St', 'address2': '', 'ad...",+17188550856,(718) 855-0856,414.846802,40.702754,-73.990884
1,60agfQbky4cX8BEApyltIA,starbucks-new-york-524,starbucks,https://s3-media4.fl.yelpcdn.com/bphoto/ZM_sCP...,False,https://www.yelp.com/biz/starbucks-new-york-52...,36,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",2.0,"{'latitude': 40.71102, 'longitude': -74.00091}",[delivery],$,"{'address1': '375 Pearl St', 'address2': '', '...",+16466999983,(646) 699-9983,833.889269,40.711020,-74.000910
2,Y_kNzTJJUx9aFj1t_DVWkA,starbucks-surrey-6,starbucks,,False,https://www.yelp.com/biz/starbucks-surrey-6?ad...,3,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",5.0,"{'latitude': 49.1186981201172, 'longitude': -1...",[],$,"{'address1': '199 Water Street', 'address2': '...",,,876.341256,49.118698,-122.895721
3,C0EKm9V9QI2R47QkRWA5aQ,starbucks-new-york-290,starbucks,https://s3-media3.fl.yelpcdn.com/bphoto/Rvok-B...,False,https://www.yelp.com/biz/starbucks-new-york-29...,44,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",3.0,"{'latitude': 40.7049014, 'longitude': -74.0072...",[],$,"{'address1': '99 Wall St', 'address2': '', 'ad...",+19299551841,(929) 955-1841,1093.379283,40.704901,-74.007244
4,OuNYQaqEJjkBHRdnC-4HOw,starbucks-new-york-381,starbucks,https://s3-media2.fl.yelpcdn.com/bphoto/SV8ynJ...,False,https://www.yelp.com/biz/starbucks-new-york-38...,40,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",2.5,"{'latitude': 40.708413, 'longitude': -74.007387}",[delivery],$$,"{'address1': '100 William St', 'address2': '',...",+12125099709,(212) 509-9709,1151.989274,40.708413,-74.007387
5,mEMPhPK6dSgy5eXS-kYHLg,starbucks-brooklyn-17,starbucks,https://s3-media4.fl.yelpcdn.com/bphoto/IsyCZQ...,False,https://www.yelp.com/biz/starbucks-brooklyn-17...,69,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",3.0,"{'latitude': 40.694582, 'longitude': -73.993978}",[delivery],$$,"{'address1': '134 Montague St', 'address2': ''...",+17182430455,(718) 243-0455,1208.186328,40.694582,-73.993978
6,t03ZbaLQP3MrMuUGb6MQdw,starbucks-new-york-506,starbucks,https://s3-media3.fl.yelpcdn.com/bphoto/XApF9R...,False,https://www.yelp.com/biz/starbucks-new-york-50...,53,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",2.0,"{'latitude': 40.704653, 'longitude': -74.009001}",[delivery],$$,"{'address1': '110 Pearl St', 'address2': '', '...",+12124826530,(212) 482-6530,1242.904106,40.704653,-74.009001
7,pTv4RDll3N_CGG3BwtEctg,starbucks-new-york-395,starbucks,https://s3-media1.fl.yelpcdn.com/bphoto/ClYnav...,False,https://www.yelp.com/biz/starbucks-new-york-39...,59,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",3.0,"{'latitude': 40.71168566, 'longitude': -74.006...",[delivery],$,"{'address1': '38 Park Row', 'address2': '', 'a...",+12126088073,(212) 608-8073,1252.899146,40.711686,-74.006669
8,j6nTfbYavGVsJKxz6bte6w,starbucks-new-york-214,starbucks,https://s3-media3.fl.yelpcdn.com/bphoto/MdLapQ...,False,https://www.yelp.com/biz/starbucks-new-york-21...,61,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",3.5,"{'latitude': 40.71022348, 'longitude': -74.007...",[delivery],$,"{'address1': '130 Fulton St', 'address2': '', ...",+12129620439,(212) 962-0439,1269.573946,40.710223,-74.007972
9,XzAZVxmGPcysK8hYdcAJKA,starbucks-new-york-229,starbucks,https://s3-media2.fl.yelpcdn.com/bphoto/ms4p2K...,False,https://www.yelp.com/biz/starbucks-new-york-22...,53,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",2.5,"{'latitude': 40.71588714, 'longitude': -74.003...",[delivery],$$,"{'address1': '111 Worth St', 'address2': '', '...",+12129648846,(212) 964-8

In [10]:
startbucks_df3 =startbucks_df2.copy()
startbucks_df3['price_point'] = startbucks_df3['price'].str.len()
startbucks_df3['price_point'] = startbucks_df3['price_point'].fillna(0)
startbucks_df3['price_point'] = startbucks_df3['price_point'].astype('int')
#startbucks_df2['price_point']

In [12]:
startbucks_df3[['address1', 'address2','address3','city','zip_code','country','state','display_address']] = pd.json_normalize(startbucks_df3['location'])
startbucks_df3

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,...,longitude,price_point,address1,address2,address3,city,zip_code,country,state,display_address
0,qcnoyytlFIuqlcjDXkXJiw,starbucks-brooklyn-21,starbucks,https://s3-media2.fl.yelpcdn.com/bphoto/rvbmfb...,False,https://www.yelp.com/biz/starbucks-brooklyn-21...,80,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",2.5,"{'latitude': 40.70275444, 'longitude': -73.990...",...,-73.990884,2,67 Main St,,,Brooklyn,11201,US,NY,"[67 Main St, Brooklyn, NY 11201]"
1,60agfQbky4cX8BEApyltIA,starbucks-new-york-524,starbucks,https://s3-media4.fl.yelpcdn.com/bphoto/ZM_sCP...,False,https://www.yelp.com/biz/starbucks-new-york-52...,36,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",2.0,"{'latitude': 40.71102, 'longitude': -74.00091}",...,-74.000910,1,375 Pearl St,,None,New York,10038,US,NY,"[375 Pearl St, New York, NY 10038]"
2,Y_kNzTJJUx9aFj1t_DVWkA,starbucks-surrey-6,starbucks,,False,https://www.yelp.com/biz/starbucks-surrey-6?ad...,3,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",5.0,"{'latitude': 49.1186981201172, 'longitude': -1...",...,-122.895721,1,199 Water Street,,,Surrey,V4E 3E4,CA,BC,"[199 Water Street, Surrey, BC V4E 3E4, Canada]"
3,C0EKm9V9QI2R47QkRWA5aQ,starbucks-new-york-290,starbucks,https://s3-media3.fl.yelpcdn.com/bphoto/Rvok-B...,False,https://www.yelp.com/biz/starbucks-new-york-29...,44,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",3.0,"{'latitude': 40.7049014, 'longitude': -74.0072...",...,-74.007244,1,99 Wall St,,,New York,10005,US,NY,"[99 Wall St, New York, NY 10005]"
4,OuNYQaqEJjkBHRdnC-4HOw,starbucks-new-york-381,starbucks,https://s3-media2.fl.yelpcdn.com/bphoto/SV8ynJ...,False,https://www.yelp.com/biz/starbucks-new-york-38...,40,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",2.5,"{'latitude': 40.708413, 'longitude': -74.007387}",...,-74.007387,2,100 William St,,,New York,10038,US,NY,"[100 William St, New York, NY 10038]"
5,mEMPhPK6dSgy5eXS-kYHLg,starbucks-brooklyn-17,starbucks,https://s3-media4.fl.yelpcdn.com/bphoto/IsyCZQ...,False,https://www.yelp.com/biz/starbucks-brooklyn-17...,69,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",3.0,"{'latitude': 40.694582, 'longitude': -73.993978}",...,-73.993978,2,134 Montague St,,,Brooklyn,11201,US,NY,"[134 Montague St, Brooklyn, NY 11201]"
6,t03ZbaLQP3MrMuUGb6MQdw,starbucks-new-york-506,starbucks,https://s3-media3.fl.yelpcdn.com/bphoto/XApF9R...,False,https://www.yelp.com/biz/starbucks-new-york-50...,53,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",2.0,"{'latitude': 40.704653, 'longitude': -74.009001}",...,-74.009001,2,110 Pearl St,,,New York,10005,US,NY,"[110 Pearl St, New York, NY 10005]"
7,pTv4RDll3N_CGG3BwtEctg,starbucks-new-york-395,starbucks,https://s3-media1.fl.yelpcdn.com/bphoto/ClYnav...,False,https://www.yelp.com/biz/starbucks-new-york-39...,59,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",3.0,"{'latitude': 40.71168566, 'longitude': -74.006...",...,-74.006669,1,38 Park Row,,,New York,10038,US,NY,"[38 Park Row, New York, NY 10038]"
8,j6nTfbYavGVsJKxz6bte6w,starbucks-new-york-214,starbucks,https://s3-media3.fl.yelpcdn.com/bphoto/MdLapQ...,False,https://www.yelp.com/biz/starbucks-new-york-21...,61,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",3.5,"{'latitude': 40.71022348, 'longitude': -74.007...",...,-74.007972,1,130 Fulton St,,,New York,10038,US,NY,"[130 Fulton St, New York, NY 10038]"
9,XzAZVxmGPcysK8hYdcAJKA,starbucks-new-york-229,starbucks,https://s3-media2.fl.yelpcdn.com/bphoto/ms4p2K...,False,https://www.yelp.com/biz/starbucks-new-york-22...,53,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",2.5,"{'latitude': 40.71588714, 'longitude': -74.003...",...,-74.003141,2,111 Worth St,,,New York,10013,US,NY,"[111 Worth St, New York, NY 10013]"


In [13]:
startbucks_df3 = startbucks_df3.drop(columns=['distance','display_phone','phone','alias','transactions','image_url', 'is_closed','url','categories','coordinates','location'])
startbucks_df3

,id,name,review_count,rating,price,latitude,longitude,price_point,address1,address2,address3,city,zip_code,country,state,display_address
0,qcnoyytlFIuqlcjDXkXJiw,starbucks,80,2.5,$$,40.702754,-73.990884,2,67 Main St,,,Brooklyn,11201,US,NY,"[67 Main St, Brooklyn, NY 11201]"
1,60agfQbky4cX8BEApyltIA,starbucks,36,2.0,$,40.711020,-74.000910,1,375 Pearl St,,None,New York,10038,US,NY,"[375 Pearl St, New York, NY 10038]"
2,Y_kNzTJJUx9aFj1t_DVWkA,starbucks,3,5.0,$,49.118698,-122.895721,1,199 Water Street,,,Surrey,V4E 3E4,CA,BC,"[199 Water Street, Surrey, BC V4E 3E4, Canada]"
3,C0EKm9V9QI2R47QkRWA5aQ,starbucks,44,3.0,$,40.704901,-74.007244,1,99 Wall St,,,New York,10005,US,NY,"[99 Wall St, New York, NY 10005]"
4,OuNYQaqEJjkBHRdnC-4HOw,starbucks,40,2.5,$$,40.708413,-74.007387,2,100 William St,,,New York,10038,US,NY,"[100 William St, New York, NY 10038]"
5,mEMPhPK6dSgy5eXS-kYHLg,starbucks,69,3.0,$$,40.694582,-73.993978,2,134 Montague St,,,Brooklyn,11201,US,NY,"[134 Montague St, Brooklyn, NY 11201]"
6,t03ZbaLQP3MrMuUGb6MQdw,starbucks,53,2.0,$$,40.704653,-74.009001,2,110 Pearl St,,,New York,10005,US,NY,"[110 Pearl St, New York, NY 10005]"
7,pTv4RDll3N_CGG3BwtEctg,starbucks,59,3.0,$,40.711686,-74.006669,1,38 Park Row,,,New York,10038,US,NY,"[38 Park Row, New York, NY 10038]"
8,j6nTfbYavGVsJKxz6bte6w,starbucks,61,3.5,$,40.710223,-74.007972,1,130 Fulton St,,,New York,10038,US,NY,"[130 Fulton St, New York, NY 10038]"
9,XzAZVxmGPcysK8hYdcAJKA,starbucks,53,2.5,$$,40.715887,-74.003141,2,111 Worth St,,,New York,10013,US,NY,"[111 Worth St, New York, NY 10013]"
